In [ ]:
# ---------------------------------------------------------------
# 03_kerr_flux_sim.ipynb  –  Kerr-enhanced quaternionic flux demo
# ---------------------------------------------------------------
# Colab cell 1 : basic setup & constants
!pip install --quiet scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, quad

# Physical constants (SI where it matters, but we work in km & s for G/c² factors)
G      = 6.67430e-11          # m³ kg⁻¹ s⁻²
c      = 2.99792458e8         # m s⁻¹
M_sun  = 1.98847e30           # kg
M_pl   = 2.176434e-8          # kg  (Planck mass)

# ---------------------------------------------------------------
# Colab cell 2 : Kerr parameters  (Sgr A*-like black hole)
M_bh   = 4.0e6 * M_sun        # 4×10⁶ M☉
a_dim  = 0.5                  # dimensionless spin (a* = J / M r_g)  ≈ 0.5
r_g_m  = 2*G*M_bh / c**2      # Schwarzschild radius in metres
r_g_km = r_g_m / 1e3          # … in km (our working unit)

a_km   = a_dim * r_g_km       # Kerr a in km

print(f"Sgr A* proxy :  r_g ≈ {r_g_km:6.2e} km,   a = {a_km:6.2e} km")

# ---------------------------------------------------------------
# Colab cell 3 : radial ODE  (dimensionless ψ = φ / M_pl)
def delta(r):        # Kerr Δ = r² − r_g r + a²  (in km²)
    return r**2 - r_g_km*r + a_km**2

beta = 1e2           # sets overall strength of λ̃(r)=β/r³  (chosen for demo)

def rhs(r, y):
    """1st-order system   y = (ψ, χ)  with  χ = dψ/dr"""
    ψ, χ = y
    lam_tilde = beta / r**3          # curvature-driven coupling
    dχ = ( r**2 * lam_tilde * ψ*(ψ**2 - 1) -  # source term
           (2/r)*r**2*χ -                # implicit derivative of r²
           ψ_prime_prefactor(r)*χ ) / delta(r)  # rearranged ODE
    return [χ, dχ]

def ψ_prime_prefactor(r):
    """helper for derivative of (r² Δ) factor"""
    return (2*r*delta(r) + r**2*(2*r - r_g_km))  # derivative of r²Δ wrt r

# ---------------------------------------------------------------
# Colab cell 4 : integrate from just outside the horizon
r_plus = (r_g_km + np.sqrt(r_g_km**2 - 4*a_km**2)) / 2
r_start = r_plus * 1.02        # 2 % outside horizon
r_end   = 100 * r_g_km         # ~ pc scale for Sgr A*

# initial tiny seed  ψ(r_start)≈0,  ψ'(r_start)=0
y0 = [1e-6, 0.0]

sol = solve_ivp(rhs, (r_start, r_end), y0, rtol=1e-9, atol=1e-12,
                dense_output=True, max_step=r_g_km/10)

r_vals = np.logspace(np.log10(r_start), np.log10(r_end), 600)
ψ_num  = sol.sol(r_vals)[0]
ε_num  = 2*ψ_num

# analytic ansatz  ε_anal = 2 tanh(r / r_s′) with Kerr boost (paper) --------------
r_s0   = 8.5e3 * 3.086e13 / 1e5    # 8.5 kpc → km  (≈ 2.6×10¹⁷ km)
boost  = 1 - a_km / r_vals         # (1 − a/r) factor
r_s_eff = r_s0 * boost
ε_anal  = 2*np.tanh(r_vals / r_s_eff)

# ---------------------------------------------------------------
# Colab cell 5 : plot ε(r)
plt.figure(figsize=(7,5))
plt.loglog(r_vals/r_g_km, ε_num, label="Numerical ε(r)", lw=2)
plt.loglog(r_vals/r_g_km, ε_anal, "--", label="Analytic ansatz", lw=2)
plt.axvline(r_plus/r_g_km, color="k", ls=":", alpha=.4)
plt.text(1.05, 0.3, "r₊", transform=plt.gca().transAxes)
plt.xlabel(r"$r / r_g$")
plt.ylabel(r"$\epsilon(r)=2\,\psi(r)$")
plt.title("Quaternionic flux in Kerr spacetime")
plt.legend(); plt.grid(alpha=.3, ls="--")
plt.tight_layout(); plt.show()

# ---------------------------------------------------------------
# Colab cell 6 : estimate graviton velocity shift  δc_g/c  near r≈10 r_g
k_LISA   = 2*np.pi*1e-2 / 3e5   # k = ω/c  (f=0.01 Hz)  → s⁻¹
idx_10   = np.argmin(np.abs(r_vals - 10*r_g_km))
ψ_10     = ψ_num[idx_10]
eps_10   = 2*ψ_10
TrH2_10  = (a_km/r_vals[idx_10])**2
deltac_over_c = (ψ_10**2 * TrH2_10) / (2*k_LISA**2)

print(f"At r≈10 r_g  :   ε ≈ {eps_10: .2e}   →   δc_g/c ≈ {deltac_over_c: .2e}")
